# Bienvenue dans le notebook consacré au pipeline final du résumé

In [2]:
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pickle

import os
os.chdir("c:\\Users\\theo.roudil-valentin\\Documents\\Codes") # A modifier, et mettre votre dossier correspondant !

try:
    os.chdir("2021-NLP_AE\\Theo")
except:
    raise ValueError("Merci de décommenter la ligne verte au-dessus, et d'indiquer correctement le chemin vers le dossier contenant le git")

from transformers import CamembertTokenizer,CamembertModel
tok=CamembertTokenizer('Model\\MLSUM_tokenizer.model')
camem=CamembertModel.from_pretrained("camembert-base")

import gensim
W2V=gensim.models.Word2Vec.load("Model\\W2V_all.model")

import fats #On importe le module contenant toutes les fonctions

## Exemple de la fonction finale de résumé !
Comme vous pouvez le constater, la fonction de résumé a beaucoup d'options, et c'est normal !
Dans l'idéal, vous pouvez sélectionner le modèle qui vous convient le mieux. J'ai essayé de laisser un maximum de choix aux utilisateurs. 

**Attention** Cette fonction est __profonde__, autrement dit, elle est basée sur un grand nombre d'autres fonctions ou suites ou classes de fonctions, qui sont toutes dans le module **fats.py** (à télécharger impérativement donc). La modification, ou même la compréhension de cette fonction et de tout ce sur quoi elle repose vous demandera du temps. J'ai, dans la mesure du possible, laisser le maximum d'indication. Les fonctions du module proviennent toutes d'un travail préliminaire, dont vous pouvez retrouver les codes dans le dossier **Exploration**

In [ ]:
# Il faut que P soit une liste de listes de phrases, donc on se débrouille pour que ce soit le cas
P=pickle.load(open('Paragraphes_exemple.pickle','rb')) #Ajoutez votre propre liste, ou décommenter les suivantes (par deux) pour exemple

#P=pickle.load(open('Paragraphes_exemple.pickle','rb'))

#P=pickle.load(open('liste.pickle','rb')) #liste de Paragraphes
#P=[i.split('.') for i in P.tolist()] # à activer si on a une liste de paragraphes qui ne soit pas de type liste (array), à commenter sinon

# On lance par exemple pour les 3 premiers éléments
resu,text_2=fats.Resume(texte=P,
                 DL=False, # True si on veut utiliser des modèles de Deep Learning, False sinon
                 cpu=1, #le nombre de cpu à utiliser, préférez peu de CPU, pour la mémoire
                 type_='TextRankBert', #le nom du modèle, si DL=False
                 k=2, #Le nombre de phrases
                 choose_model=None, #le nom du modèle de Deep Learning, le cas échéant
                 tok='Model\\MLSUM_tokenizer.model', #le nom du tokenizer
                 modele=camem, #modèle CamemBERT ou W2V selon le modèle choisi
                 get_score_only=False,# est-ce qu'on veut juste le score et pas directement les phrases
                 s=True,vs=12000,sp=1,tr=False,t=True,seuil=2,lem=False,sc=3,
                 weights=True,alpha=0.2,frac=0.25)
print(resu) 